In [2]:
import pandas as pd
import numpy as np
import os
import string
import glob
import random
import cmath
import math

In [4]:
Dimen=1000
Ngram=3

In [25]:
#########Loading data##################

LangLabels = ['bul', 'ces', 'dan', 'nld', 'deu', 'eng', 'est', 'fin', 'fra', 'ell', 'hun', 'ita',
              'lav', 'lit', 'pol', 'por', 'ron', 'slk', 'slv', 'spa', 'swe']
Traindata=[]
Trainlabel=[]
for i, Label in enumerate(LangLabels):
    FilePath = os.path.join('language','training_texts',"{}.txt".format(Label))
    with open(FilePath,'r', encoding='utf-8') as f:
        inputseq=f.read()
    senlist=inputseq.split('\n')
    for data in senlist:
        Traindata.append(data)
        Trainlabel.append(i)
print("{} sentences have been loaded for train".format(str(len(Traindata))))

210080 sentences have been loaded for train


In [26]:
ShortLangLabels = ['bg', 'cs', 'da', 'nl', 'de', 'en', 'et', 'fi', 'fr', 'el', 'hu', 'it', 'lv', 'lt', 'pl', 'pt', 'ro', 'sk', 'sl', 'es', 'sv']
Testdata=[]
Testlabel=[]
mp = dict()
for i,key in enumerate(ShortLangLabels):
    mp[key] = i
TestFilePathList = glob.glob(os.path.join("language", "testing_texts", "*.txt"))
for i, TestFilePath in enumerate(TestFilePathList):
    (_, TestFileName) = os.path.split(TestFilePath)
    if not mp.__contains__(TestFileName[0:2]):
        continue
    Label = mp[TestFileName[0:2]]
    with open(TestFilePath,'r',encoding='utf-8') as f:
        InputSequence = f.read()
    Testdata.append(InputSequence[:-1]) 
    Testlabel.append(Label)
print("{} sentences have been loaded for test.".format(str(len(Testdata))))

21000 sentences have been loaded for test.


In [27]:
#Cartesian
K=21
IM = dict() 
AM = dict()
IAM = dict()
LangLabels = []
for key in range(K):
    AM[key] = np.zeros(Dimen)
    LangLabels.append(key)

def GenerateHV():
    real_part = np.random.uniform(low=-1, high=1, size=Dimen)
    complex_part = np.random.uniform(low=-1, high=1, size=Dimen)
    RandomHV = real_part + 1j * complex_part
    return RandomHV

def CalculateCosineSimilarity(vector1, vector2):
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)

    norm1 = np.sqrt(np.dot(vector1, np.conjugate(vector1)))
    norm2 = np.sqrt(np.dot(vector2, np.conjugate(vector2)))
    return np.real(np.dot(vector1, np.conjugate(vector2)) / (norm1*norm2))

def LookupIM(Key):
    if not IM.__contains__(Key):
        IM[Key] = GenerateHV();
    return IM[Key]

def NGramEncoding(InputSequence):
    block = np.zeros([Ngram, Dimen])
    sumHV = np.zeros(Dimen)
    for i, Key in enumerate(InputSequence):
        block = np.roll(block, (1, 1), (0, 1))
        block[0] = LookupIM(Key)
        if i >= Ngram - 1:
            NGramHV = block[0]
            for j in range(1, Ngram):
                NGramHV = NGramHV * block[j]
            sumHV = sumHV + NGramHV
    return sumHV


length = len(Traindata)
for i in range(length):
    Data = Traindata[i]
    Label = Trainlabel[i]
    tmp = NGramEncoding(Data)
    AM[Label] = AM[Label] + tmp
for Label in LangLabels:
    IAM[Label] = AM[Label]


C:\Users\mdasi\AppData\Local\Temp\ipykernel_14108\360643286.py:35: ComplexWarning: Casting complex values to real discards the imaginary part
  block[0] = LookupIM(Key)


In [28]:
length = len(Testdata)
predict = []
for i in range(length):
    Data = Testdata[i]
    TestHV = NGramEncoding(Data)
    maxAngle = -1
    predicLang = -1
    for j,LangLabel in enumerate(LangLabels):
        angle = CalculateCosineSimilarity(AM[LangLabel], TestHV)
        if angle > maxAngle:
            maxAngle = angle
            predicLang = LangLabel
    predict.append(predicLang)

C:\Users\mdasi\AppData\Local\Temp\ipykernel_14108\360643286.py:35: ComplexWarning: Casting complex values to real discards the imaginary part
  block[0] = LookupIM(Key)


In [29]:
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
print("Accuracy score: ",accuracy_score(Testlabel,predict))

Accuracy score:  0.9594761904761905


In [9]:
#Ciclelookup
class CircleLookupTable:
    PHASE_RESOLUTION=16384
    PI=math.pi
    HALF_PI=PI/2
    STEPS_PER_RADIAN = PHASE_RESOLUTION / (2*PI)
    RADIANS_PER_STEP = (2*PI) / PHASE_RESOLUTION
    ZERO_INDEX=-1
    realLUT=[]
    imagLUT=[]
    
    def __init__(self):
        self.realLUT = np.empty(PHASE_RESOLUTION,dtype=object)
        self.imagLUT = np.empty(PHASE_RESOLUTION,dtype=object)
    
    def initialize(self):
        singletonInstance = CircleLookupTable()
        singletonInstance.realLUT[0]=0
        singletonInstance.imagLUT[0]=0
        for i in range(PHASE_RESOLUTION):
            theta = i*RADIANS_PER_STEP
            singletonInstance.realLUT[i]=math.cos(theta)
            singletonInstance.imagLUT[i]=math.sin(theta)
    def getRealEntry(self,i):
        if(i==ZERO_INDEX):
            return 0
        if (singletonInstance ==None):
            initialize()
        return singletonInstance.realLUT[i]
    def getImagEntry(self,i):
        if(i==ZERO_INDEX):
            return 0
        if (singletonInstance ==None):
            initialize()
        return singletonInstance.imagLUT[i]
        
    def phaseAngleFromCartesianTrig(self, real, imag):
        if(real==0 and imag==0):
            return ZERO_INDEX
        theta=HALF_PI-math.atan2(real,imag)
        steps=theta*STEPS_PER_RADIAN
        if(steps<0):
            steps+=PHASE_RESOLUTION
        return steps


x=CircleLookupTable()
x.realLUT[0]

NameError: name 'PHASE_RESOLUTION' is not defined

In [ ]:
#Polar dense 
phaseAngles=[-1 for i in range(Dimen)]
for i in range(Dimen):
    testPlace=np.random.permutation(Dimen)
    randomPhaseAngle= np.random(CircleLookupTable.PHASE_RESOLUTION)
    
    

In [5]:
real_part = np.random.uniform(low=-1, high=1, size=Dimen)
complex_part = np.random.uniform(low=-1, high=1, size=Dimen)
complex_part=1j*complex_part
RandomHV1 = real_part + complex_part

In [6]:
real_part = np.random.uniform(low=-1, high=1, size=Dimen)
complex_part = np.random.uniform(low=-1, high=1, size=Dimen)
complex_part=1j*complex_part
RandomHV2 = real_part + complex_part

In [7]:
x=[cmath.phase(i) for i in RandomHV1]
y=[cmath.phase(i) for i in RandomHV2]

In [8]:
y

[-0.7419228462157643,
 -1.4851717708087149,
 1.637085481530996,
 -2.4140618290781144,
 2.081200840557524,
 2.340745072489362,
 -0.7044917054548918,
 2.7399169214491343,
 2.3122044971511917,
 -1.9783440658878433,
 -0.5680900816472984,
 1.0143230459752834,
 -0.25186671735179156,
 1.7075378389437614,
 0.5994532869307352,
 -1.5906954583138315,
 -0.3074506646047195,
 -2.419457955792323,
 0.5053568272369541,
 1.0097879302917203,
 0.7815932914239115,
 1.4891164750057486,
 2.0261486276814593,
 -0.6557171789437299,
 -2.3816157285514397,
 0.09360734475791248,
 2.0635795437082347,
 -3.0921097369046695,
 -1.0647010904959042,
 -3.0727675655094036,
 2.3375081251777514,
 -2.4592582310178774,
 -2.3169904191144135,
 3.0715640938754345,
 -0.8974295462427418,
 -0.7048724709399581,
 0.5161188008593702,
 -0.7019701885509352,
 -2.3010329654569657,
 0.20080152773951127,
 -2.2683723863866345,
 -0.6109311818841628,
 -0.6720136586483465,
 -1.440265676064426,
 2.0517646587230747,
 2.7884351075070017,
 -3.0229419

In [11]:
Norm1 = np.linalg.norm(x)
Norm2 = np.linalg.norm(y)
(abs(Norm1-Norm2))/1000

0.0008431436973477631

In [10]:
sum=0
xx=0
for i,j in zip(x,y):
    sum+=abs(i-j)
sum/1000

2.152399243442055

In [105]:
sum

1988.8418107023692

In [86]:
for i,j in zip(x,y):
    sum=i-j
    print(sum)

0.3799572180745848
-1.9924946356266793
2.1059157649237914
1.941373225592939
3.8878879171506773
-2.1668714993481406
0.7905011955921929
2.256833940792494
-1.686100881075994
-0.577420718297859
-0.500340495689082
0.7251790965978261
-3.7663532376732913
4.311369108797713
-4.198811150470884
3.5958752773404568
3.4545158987171125
-0.954251247537881
-2.7612909708228432
1.2625871297535933
0.7996182693529856
3.084324161394873
4.446206987328788
0.5080863935914164
-1.7154962883732896
-4.775559450787693
-1.7688419322071058
0.08671238372285317
-5.535093706028869
3.5543031993311134
1.182243100265955
4.701873182599876
-0.016903452784690298
-3.122683185454364
3.370055472416407
-0.6697952305711717
0.7037653001199085
-2.39809432732035
0.9583952247238474
-2.1686889623652665
1.8772710699655444
5.189329508573431
0.9897300008975711
-1.6578278849115127
1.7540247573853716
1.7355581553943245
0.5025848803974209
-1.420486341515931
2.080314967262933
4.31434028537254
-0.3872854915031798
-1.4022746654933826
2.74133222

IndexError: invalid index to scalar variable.

In [93]:
vector1 = np.linalg.norm(x)
vector2 = np.linalg.norm(y)

vector2

57.284000881316516

In [29]:
randvec = np.random.uniform(low=-.5/Dimen, high=.5/Dimen, size=Dimen)
randvec

array([ 1.33298004e-04,  1.13511538e-04, -7.87002525e-05, -1.06900963e-04,
       -3.03742285e-04, -1.51023185e-04, -1.39945351e-04, -3.82144456e-04,
        1.71988508e-04,  1.43838745e-04,  2.38735057e-04,  4.14032257e-04,
        4.86399496e-05, -4.75309303e-04,  4.98506980e-04,  2.76212760e-04,
       -1.59879216e-05,  3.37253871e-04,  1.02767706e-04, -2.34003733e-05,
        2.19988744e-04, -2.04478640e-04,  9.83312196e-05,  1.89430441e-04,
       -3.86676281e-04,  3.33172309e-04,  4.04678463e-04,  7.80426973e-05,
        1.10123856e-04, -4.31336260e-05,  2.87731924e-04,  3.84495408e-04,
       -2.77294695e-04, -4.13939251e-04, -4.06377977e-04,  4.66265975e-04,
       -2.67022058e-05,  2.96824609e-05, -3.93808496e-05, -2.85399427e-04,
       -2.20390705e-04,  2.11205290e-04,  1.22761036e-04,  2.39141415e-04,
       -3.82278244e-04,  2.49819715e-04,  1.37382228e-04, -4.17715062e-05,
        3.10726522e-04, -1.72549525e-05,  4.66241112e-04,  1.00536129e-04,
        4.29775791e-04,  